Use 25 ETFs from 5 asset classes. ETFs chosen for each asset classes are based on the market capitalization size. Pick 3 ETFs with strongest 6 month momentum into portfolio and weight them equally. Hold for 1 month and then rebalance.

US Equities (S&P 500) = SPY, VOO, IVV

Bonds (Total Bond Market) = AGG, BND, BNDX

REITs = VNQ (North America), VNQI (Global excluding US), REET (Global)

Foreign Equities (Foreign Large Cap) = VEA, IEFA, EFA

Commodities (Broad Diversified) = DBC, PDBC, GSG

*Ranking based on etfdb.com

Original Source : https://quantpedia.com/strategies/asset-class-momentum-rotational-system/

Modifications : Reduced period from 12M momentum to 6M to capture better, expanded universe from 5 ETFs to 25 ETFs 



In [ ]:
import pandas as pd
from datetime import datetime

class AssetClassMomentumAlgorithm(QCAlgorithm):

    def Initialize(self):

        self.SetStartDate(2001, 1, 1)  
        self.SetEndDate(2020, 12, 31)  
        self.SetCash(100000)
        
        # create a dictionary to store momentum indicators for all symbols 
        self.data = {}
        period = 6*21
        self.symbols = ["SPY", "VOO", "IVV", "AGG", "BND", "BNDX" 
                        "VNQ", "VNQI", "REET", "VEA", "IEFA", "EFA", 
                        "DBC", "PDBC", "GSG"]
        
        # warm up the MOM indicator
        self.SetWarmUp(period)
        for symbol in self.symbols:
            self.AddEquity(symbol, Resolution.Daily)
            self.data[symbol] = self.MOM(symbol, period, Resolution.Daily)
            
        # schedule the function to fire at the month start 
        self.Schedule.On(self.DateRules.MonthStart("SPY"), self.TimeRules.AfterMarketOpen("SPY"), self.Rebalance)
            
    def OnData(self, data):
        pass

    def Rebalance(self):
        if self.IsWarmingUp: return
        top5 = pd.Series(self.data).sort_values(ascending = False)[:5]
        for kvp in self.Portfolio:
            security_hold = kvp.Value
            # liquidate the security which is no longer in the top5 momentum list
            if security_hold.Invested and (security_hold.Symbol.Value not in top5.index):
                self.Liquidate(security_hold.Symbol)
        
        added_symbols = []        
        for symbol in top5.index:
            if not self.Portfolio[symbol].Invested:
                added_symbols.append(symbol)
        for added in added_symbols:
            self.SetHoldings(added, 1/len(added_symbols))